In [23]:
  !git clone -q https://github.com/fsiraj/Vegetation-Encroachment-on-Power-Infrastructure
  !mv ./Vegetation-Encroachment-on-Power-Infrastructure/* ./
  !rm -rf sample_data
  !rm -rf Vegetation-Encroachment-on-Power-Infrastructure

In [24]:
!pip install -q geopandas geemap

In [25]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import datetime
import pprint

In [26]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=awYMw_oEdwWyp32etlaBxbkIxyIoVZMFIIGVo5j73LM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWilmHO9yURFk9D3xsNddHGFu9BlQkySoxjeUCv8mCHNYE1JYpZPc9g

Successfully saved authorization token.


In [27]:
ee.Initialize()

In [28]:
saoMiguel = ee.Geometry.Point([-25.3425, 37.7532])
boundary = geemap.kml_to_ee('./study_area.kml').geometry()

# Build Sentinel Collection

In [29]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [30]:
AOI = saoMiguel
START_DATE = '2018-01-01'
END_DATE = '2021-07-30'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 60
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 10

In [31]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [32]:
def add_ndvi(image):
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('ndvi'))

In [33]:
sen2 = get_s2_sr_cld_col(AOI, START_DATE, END_DATE).map(add_cld_shdw_mask).map(apply_cld_shdw_mask).map(add_ndvi)

# Actual Machine Learning

In [34]:
Map = geemap.Map(center=[37.7529, -25.3453], zoom=10)
#training = ee.Image('COPERNICUS/CORINE/V20/100m/2012')
training_shp = './training_labels.shp'
training = geemap.shp_to_ee(training_shp)
landCover = training.select('landcover')
Map.addLayer(landCover, {}, 'Land cover')
Map

In [35]:
# Load Sentinel data
image = sen2.sort('CLOUD_COVER').first()
trueColour = {
    'bands': ["B4", "B3", "B2"],
    'min': 0,
    'max': 3000
}

bands = ['B2', 'B3', 'B4', 'B8']
input = image.select(bands)

In [36]:
# Load training data
training = ee.FeatureCollection('COPERNICUS/CORINE/V20/100m')
label = 'landcover'

# Overlay training data on imagery
trainImage = input.sampleRegions(**{
    'collection': training,
    'properties': [label]
})

# Train-Test split of the training data
trainingData = trainImage.randomColumn()
trainSet = trainingData.filter(ee.Filter.lessThan('random', 0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', 0.8))

In [37]:
# Random Forest Classifier
classifier = ee.Classifier.smileRandomForest(10).train(**{
    'features': trainSet,
    'classProperty': label,
    'inputProperties': bands
})

# Classify the image
classified = input.classify(classifier)

In [38]:
# Get information about the bands as a list.
band_names = classified.bandNames()
print('Band names:', band_names.getInfo())  # ee.List of band names

EEException: ignored